In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

%matplotlib inline

In [3]:
df=pd.read_csv(r'C:\Users\PallaviM777\Downloads\lixpm28.csv')
df_test=pd.read_csv(r'C:\Users\PallaviM777\Downloads\lixpm28.csv')

#dataframes for target and non target classes
df_yes=df.loc[df['y']=='yes']
df_no=df.loc[df['y']=='no']

cols= list(df.columns[1:5])
[cols.append(i) for i in list(df.columns[6:9])]

#converting categorical values to numeric to evaluate correlation
ctr=1
for i in df_test['job'].unique():
    df_test.loc[df_test['job']==i,'job']=ctr
    ctr+=1
ctr=1

for i in df_test['marital'].unique():
    df_test.loc[df_test['marital']==i,'marital']=ctr
    ctr+=1
ctr=1

for i in df_test['education'].unique():
    df_test.loc[df_test['education']==i,'education']=ctr
    ctr+=1
ctr=1

for i in df_test['contact'].unique():
    df_test.loc[df_test['contact']==i,'contact']=ctr
    ctr+=1

ctr=1

for i in df_test['poutcome'].unique():
    df_test.loc[df_test['poutcome']==i,'poutcome']=ctr
    ctr+=1

df_test.loc[df_test['default']=='yes','default']=1
df_test.loc[df_test['default']=='no','default']=0

df_test.loc[df_test['housing']=='yes','housing']=1
df_test.loc[df_test['housing']=='no','housing']=0

df_test.loc[df_test['loan']=='yes','loan']=1
df_test.loc[df_test['loan']=='no','loan']=0

df_test.loc[df['y']=='yes','y']=1
df_test.loc[df['y']=='no','y']=0


In [ ]:
df_yes.describe()

In [ ]:
df_no.describe()

In [ ]:
df_yes[cols].describe(include='all')

In [ ]:
df_no[cols].describe(include='all')

In [ ]:
#Correlation heatmap
sns.set()
corr=df_test.corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns, cmap="YlGnBu")
plt.show()

In [ ]:
#target/non target class count
labels = ['yes','no']
values = [len(df_yes), len(df_no)]
plt.bar([0, 1], values, align='center', color=['blue', 'orange'])
plt.xticks([0, 1], labels)
plt.show()

In [ ]:
pinfo=list(df.columns[2:4])
pinfo.append(df.columns[8])

fininfo=list(df.columns[6:8])
fininfo.append(df.columns[4])

numinfo = []
numinfo.append(df.columns[0])
numinfo.append(df.columns[5])
numinfo.append(df.columns[9])

campaigninfo=list(df.columns[11:14])

In [ ]:
#Comparison plots for demographics

plt.rcParams['figure.figsize'] = (18,5.0)

fig, axes = plt.subplots(ncols=3,nrows=1)
sns.set()
sns.set_palette("Paired")
for ax, col in zip(axes.flat, pinfo):
    chart= sns.countplot(x=col,data=df[[col,'y']], hue='y', ax=ax)
    chart.set_xticklabels(chart.get_xticklabels(),rotation=90)
    chart.set_title('Comparison based on '+col)
plt.tight_layout()
plt.show()

In [ ]:
#Comparison plots for financial info

plt.rcParams['figure.figsize'] = (20.0, 5.0)

fig, axes = plt.subplots(ncols=3, nrows=1)
sns.set()
sns.set_palette("Paired")
for ax, col in zip(axes.flat, fininfo):
    chart= sns.countplot(x=col,data=df[[col,'y']], hue='y', ax=ax)
    chart.set_xticklabels(chart.get_xticklabels(),rotation=90)
    chart.set_title('Comparison based on '+col)
plt.show()

In [ ]:
#Poutcome plot

plt.rcParams['figure.figsize'] = (5.0, 5.0)
sns.set()
sns.set_palette("Paired")
chart= sns.countplot(x='poutcome',data=df[['poutcome','y']], hue='y')
chart.set_xticklabels(chart.get_xticklabels(),rotation=90)
chart.set_title('Comparison based on Poutcome')
plt.show()

In [ ]:
#Job plot

plt.rcParams['figure.figsize'] = (5.0, 5.0)
sns.set()
sns.set_palette("Paired")
chart= sns.countplot(x='job',data=df[['job','y']], hue='y')
chart.set_xticklabels(chart.get_xticklabels(),rotation=90)
chart.set_title('Comparison based on job')
plt.show()

In [ ]:
#Housing plot

plt.rcParams['figure.figsize'] = (5.0, 5.0)
sns.set()
sns.set_palette("Paired")
chart= sns.countplot(x='housing',data=df[['housing','y']], hue='y')
chart.set_xticklabels(chart.get_xticklabels(),rotation=90)
chart.set_title('Comparison based on housing')
plt.show()

In [ ]:
#Numeric feature plot

d=df.groupby(['y'])
plt.rcParams['figure.figsize'] = (20.0, 5.0)
fig, axes = plt.subplots(ncols=3, nrows=1)
sns.set()
sns.set_palette("Paired")
for ax,col in zip(axes.flat,numinfo):
    value=d[col].median()
    chart=value.plot(kind='bar', color='blue', ax=ax)
    chart.set_title('Comparison based on '+col)
plt.show()

In [ ]:
#Balance plot

d=df.groupby(['y'])
plt.rcParams['figure.figsize'] = (5.0, 5.0)
sns.set()
sns.set_palette("Paired")
value=d['balance'].median()
chart=value.plot(kind='bar', color='blue')
chart.set_title('Comparison based on balance')
plt.show()

In [ ]:
#Campaign plots

d=df.groupby(['y'])
plt.rcParams['figure.figsize'] = (20.0, 5.0)
fig, axes = plt.subplots(ncols=3, nrows=1)
sns.set()
sns.set_palette("Paired")
for ax,col in zip(axes.flat,campaigninfo):
    value=d[col].median()
    chart=value.plot(kind='bar', color='blue', ax=ax)
    chart.set_title('Comparison based on '+col)
plt.show()

In [ ]:
X=df_test[df_test.columns[:-1]]
Y=df_test['y']
dt = tree.DecisionTreeClassifier(max_depth=20, min_samples_leaf=7)
folds = KFold(n_splits=5, shuffle=True, random_state=42)
dt_scores = cross_val_score(dt, X, Y, scoring='precision', cv=folds)
print("Mean DT Accuracy:", np.mean(dt_scores))

dt.fit(X,Y)
feature_imp=dict(zip(X.columns, dt.feature_importances_))
sns.set(style="whitegrid")
ax = sns.barplot(x=X.columns, y=dt.feature_importances_, color='orange')
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
plt.show()

In [ ]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X,Y)
for i,j in zip(df.columns,sel.get_support()):
    print(i,j)